In [16]:
import os
import torch
import glob
from torchvision import models, transforms
from PIL import Image, ImageDraw
import torch.nn as nn
from annoy import AnnoyIndex
import pandas as pd

In [4]:
database_images= glob.glob('../db/*/*.jpg')
len(database_images)

41

In [5]:
db_dict ={
    '1000':'7-oil-red', '1001':'7-oil-green', '1002':'7-oil-black', '1003':'7-oil-brown', 
    '1004':'7-oil-yellow', '1005':'7-oil-orange', '1006':'fathima-kesha-wardhani', 
    '1007':'nawarathna-oil-box-green','1008':'nawarathna-oil-box-red', '1009':'janet-hair-fall-red', 
    '1010':'janet-hair-fall-blue', '1011':'bread-growth', '1012':'7-oil-white', '1013':'castor-oil', '1014':'hair-care-oil-blue', '1015':'jasmin-coconut-hari-oil',
    '1016':'chandanalepa-box', '1017':'pears-baby-cream','1018':'parachuti-hail-oil', '1019':'amla-hurbal-hail-oil',
    '1020':'janet-hair-fall-green'
          }

In [6]:
transform = transforms.Compose(
    [
        transforms.Resize((640,640)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]    
)

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [8]:
model = models.mobilenet_v2(pretrained=False).to(device)
model.classifier = torch.nn.Sequential(
    torch.nn.Dropout(p=0.2, inplace=False),
    torch.nn.Linear(in_features=1280, out_features=192, bias=True)
).to(device)
model.load_state_dict(torch.load('mobilenet_transfer_learning.pth', map_location=device))

d:\YoloV8\Product_Segmentation_YoloV8\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\YoloV8\Product_Segmentation_YoloV8\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


<All keys matched successfully>

In [9]:
model.classifier = torch.nn.Identity()
model.eval()

MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [10]:
annoy_index = AnnoyIndex(1280, 'angular')

In [11]:
for i in range(len(database_images)):
    # image_path = os.path.join(image_folder, images[i])
    image = Image.open(database_images[i])
    # print(image)
    image_tensor = transform(image).unsqueeze(0)
    output_tensor = model(image_tensor)
    annoy_index.add_item(i, output_tensor[0])

#     if i%100 == 0:
#         print(f'{i}th image proceeds')

annoy_index.build(10)
annoy_index.save('productDbIndex_MobileNetV2_trained_model.ann')

True

In [12]:
test_images_folder = '../cropped_segments'
test_images = os.listdir(test_images_folder)
len(test_images)

190

In [13]:
annoy_index = AnnoyIndex(1280, 'angular')
annoy_index.load('productDbIndex_MobileNetV2_trained_model.ann')

True

In [14]:
image_grid = Image.new('RGB', (400,200))

In [15]:
lis = []
for i in range(len(test_images)):
    image_path = os.path.join(test_images_folder, test_images[i])
    image = Image.open(image_path)
    image_tensor = transform(image).unsqueeze(0)
    output_tensor = model(image_tensor)

    nns, distance = annoy_index.get_nns_by_vector(output_tensor[0], 1, include_distances=True)
    lis.append([image_path, nns[0],distance[0]])

    image = image.resize((200,200))
    image_draw = ImageDraw.Draw(image)
    image_draw.rectangle([(0,0),(199,199)], outline='red', width=8)
    image_grid.paste(image, ((0,0)))

    search_image = Image.open(database_images[nns[0]]).resize((200,200))
    image_grid.paste(search_image, ((200,0)))
    image_grid.save(f'../dumpImage/image_{i}_{distance}.png')

In [17]:
df = pd.DataFrame(lis, columns=['testImagePath','position', 'distance'])

def product_name(position):
    key = database_images[position].split('\\')[1]
    return db_dict[key]

df['productName'] = df['position'].apply(product_name)

df.head()

,testImagePath,position,distance,productName
0,../cropped_segments\segmented_region_only_0_0.jpg,36,0.237357,parachuti-hail-oil
1,../cropped_segments\segmented_region_only_0_1.jpg,4,0.296850,7-oil-black
2,../cropped_segments\segmented_region_only_0_10...,5,0.444315,7-oil-black
3,../cropped_segments\segmented_region_only_0_11...,25,0.477211,7-oil-white
4,../cropped_segments\segmented_region_only_0_12...,2,0.478610,7-oil-green


In [18]:
counntDf = pd.DataFrame(df[df['distance']<0.45].productName.value_counts())
counntDf.reset_index(inplace=True)
counntDf

,productName,count
0,7-oil-green,9
1,parachuti-hail-oil,5
2,jasmin-coconut-hari-oil,5
3,7-oil-black,4
4,7-oil-yellow,4
5,7-oil-brown,4
6,amla-hurbal-hail-oil,3
7,7-oil-orange,3
8,fathima-kesha-wardhani,3
9,7-oil-red,2
